# Produce time-ordered files for BAM-SIF and SM1/2 data

In [15]:
import glob
import re as regex

In [16]:
dataroot = "../datasets/sif_fits/"  # all the SIF files are here (simlink)

In [17]:
# grab the paths
sm1path = glob.glob(dataroot+"*/*SM1*CDP_NONE.fits")  # There are a few CLOCKING ones there - can I use them?
sm2path = glob.glob(dataroot+"*/*SM2*CDP_NONE.fits")
bampath = glob.glob(dataroot+"*/*BAM*CLOCKING*")

# remove the ../
sm1path = [p[3:] for p in sm1path]
sm2path = [p[3:] for p in sm2path]
bampath = [p[3:] for p in bampath]

# sort them
# problem: just applying the standard "sorted" function does not work
# -> it sorts by date, but then picks up on the ROW, I need OBMT!

getobmt = lambda x: int(regex.search("(?<=OBMT_START_)[0-9]+",x).group(0))

sm1path.sort(key=getobmt)
sm2path.sort(key=getobmt)
bampath.sort(key=getobmt) # note: I think this contains some stuff besides our images with patterns

In [18]:
# write everyting into files

filenames = ["filenames_sm1.txt", "filenames_sm2.txt", "filenames_bam-sif.txt"]
pathlists = [sm1path, sm2path, bampath]

for ii in range(len(pathlists)):
    file = open(filenames[ii], 'w')
    pathlist = pathlists[ii
                        ]
    for path in pathlist:
        file.write("%s\n" % path)
        
    file.close()